In [5]:
# %load Base User.py
import gc
import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF

data = pd.read_csv('../data/UserMovie_train.txt',sep='\t')

In [7]:
# uid 48613  mid 72541
mid = data['mid'].unique()
uid = data['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data['mid'] = data['mid'].map(lambda x:collect_mid_dencoder[x])
data['uid'] = data['uid'].map(lambda x:collect_uid_dencoder[x])

In [3]:
user_matrix = np.zeros([len(uid),len(mid)])
item_matrix = np.zeros([len(mid),len(uid)])
data.head(5)

MemoryError: 

In [8]:
to_make = np.array(data[['uid','mid','star']].values,dtype=np.uint32)

In [9]:
to_make

array([[    0,     0,     5],
       [    1,     1,     4],
       [    2,     2,     5],
       ...,
       [48764,  1049,     4],
       [48683,   504,     5],
       [48699,  2038,     5]], dtype=uint32)

In [10]:
#构建用户-物品评分矩阵
to_make = np.array(data[['uid','mid','star']].values,dtype=np.uint32)
for i in range(len(to_make)):
    if i%1000000 == 0:
        print(i)
    user_matrix[to_make[i][0],to_make[i][1]] = to_make[i][2]
    item_matrix[to_make[i][1],to_make[i][0]] = to_make[i][2]

0


NameError: name 'user_matrix' is not defined

###### user_matrix.shape

In [59]:
split_test = user_matrix[:100]
split_train = user_matrix[100:]
print(split_train.shape)
print(split_test.shape)

(48513, 72540)
(100, 72540)


In [48]:
# import random
# new_vector = np.zeros(len(mid)) # readline
# for i in range(200000):
#     kfc = random.randint(1,len(uid))
#     new_vector[kfc] = random.randint(0,5)
    
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    
def calc_sim(vec,user_matrix):
    ans = []
    for i in range(len(user_matrix)):
        if i % 10000 ==0:
            print(i)
        #ans.append(np.linalg.norm(vec-user_matrix[i]))
        ans.append(cos_sim(vec,user_matrix[i]))
    return ans

In [67]:
import random
now = 0
for new_vec in split_test:
    ans = calc_sim(new_vec,split_train)
    to_recommand = DF(ans).reset_index().rename(columns={0:'sim'})
    k = 20
    topk = list(to_recommand.sort_values(by=['sim'],ascending=False).head(k)['index'].values)
    no_to_use = []
    for i in range(len(new_vec)):
        if random.randint(0,100) % 10 == 0:
            if new_vec[i]!=0:
                no_to_use.append(i)
    # 推荐没给他看过的
    watch_set = set()
    for i in topk:
        watch_set = watch_set|set(data[data['uid']==i]['mid'].unique())

    watch_set = set(watch_set)-set(no_to_use)
    print('The New One No Watch Movie Length: ',len(watch_set))
    topN = []
    for i in list(watch_set):
        temp = 0
        for j in topk:
            temp += to_recommand[to_recommand['index']==j]['sim'].values*user_matrix[j][i]
        topN.append(temp)

    topN = DF(topN).rename(columns={0:'score'})
    topN['mid'] = watch_set
    need_to_show = 20
    show_video = topN.sort_values(by=['score'],ascending=False).head(need_to_show)['mid'].values
    true_video = [collect_mid_encoder[i] for i in show_video]
    print(topN.sort_values(by=['score'],ascending=False).head(need_to_show))
    print('True: ',data[(data['mid'].isin(show_video) & (data['uid']==now))]['star'].unique())
    now += 1
    break

0
10000
20000
30000
40000
The New One No Watch Movie Length:  4825
          score   mid
528   12.586676   723
411   11.525974   563
20    11.130851    26
881   11.012054  1234
149   10.337245   205
219   10.232550   289
755   10.229440  1040
341   10.156449   454
100    9.939907   127
131    9.716812   172
588    9.704327   803
169    9.396420   233
655    9.341401   909
461    9.246755   630
1219   9.234132  1740
1526   9.175006  2295
141    8.950555   192
111    8.950059   141
696    8.929208   961
688    8.723942   949
True:  [5. 4. 3.]


In [63]:
data[data['uid']==0][['mid','star']]

,mid,star
0,0,5.0
67430,512,5.0
79543,6238,4.0
120946,949,5.0
168477,1128,5.0
280855,1123,3.0
334701,1366,5.0
376758,5549,5.0
396220,1503,4.0
497489,687,4.0


In [56]:
for i in range(len(user_matrix[0])):
    if user_matrix[0][i]!=0:
        print(i)

0
53
79
80
93
113
126
141
145
152
162
172
176
192
196
225
266
268
278
281
289
307
313
347
354
357
358
365
370
410
430
454
461
500
512
515
537
562
563
596
603
615
616
620
622
629
630
649
654
674
687
688
723
738
753
758
794
818
838
849
864
928
949
961
980
1014
1019
1040
1119
1123
1128
1146
1150
1161
1302
1324
1363
1366
1371
1450
1480
1503
1536
1684
1845
1886
1896
1921
1981
2035
2044
2324
2365
2367
2391
2470
2637
2638
2685
2825
2917
2935
2966
3041
3136
3168
3237
3250
3279
3287
3376
3444
3509
3522
3544
3562
3590
3597
3642
3647
3663
3769
3783
3793
3828
3943
4072
4544
4576
4744
5209
5304
5549
6238
6433
6629
8270
8522
11900
16354


In [58]:
# data.sort_values(by=['uid','mid'])
data[data['uid']==0].nunique()

uid            1
mid          140
timeStamp     63
star           3
dtype: int64